In [1]:
from pathlib import Path
from echoflow import echoflow_start, glob_url
from urllib.parse import urlparse
import fsspec
from typing import Dict, Optional, Any, Union, Tuple, List

dataset_config = Path("./datastore.yaml").resolve()
pipeline_config = Path("./pipeline.yaml").resolve()

In [2]:
all_files = glob_url("s3://ncei-wcsd-archive/data/raw/Bell_M._Shimada/SH1707/EK60/*.raw", {'anon':True})

In [3]:
files = []
for file in all_files:
    f = file.split(".r")[0]
    files.append(f.split("/")[-1])

In [4]:
transect = open('EK60_SH1707_Shimada.txt','w')
i = 0
for f in files:
    if i == 100:
        break
    transect.write(f+".raw\n")
    i = i + 1
transect.close()

In [5]:
options = {"storage_options_override": False}
data  = echoflow_start(dataset_config=dataset_config, pipeline_config=pipeline_config, options=options)

<Client: 'tcp://127.0.0.1:43583' processes=5 threads=5, memory=15.61 GiB>
--------------------------------------------------

Executing stage :  name='echoflow_open_raw' module='echoflow.stages.subflows.open_raw' external_params=None options={'save_raw_file': True, 'use_raw_offline': True, 'use_offline': True} prefect_config=None
[Errno 111] Connection refused

Completed stage name='echoflow_open_raw' module='echoflow.stages.subflows.open_raw' external_params=None options={'save_raw_file': True, 'use_raw_offline': True, 'use_offline': True} prefect_config=None
--------------------------------------------------
<Client: 'tcp://127.0.0.1:43583' processes=5 threads=5, memory=15.61 GiB>
--------------------------------------------------

Executing stage :  name='echoflow_compute_TS' module='echoflow.stages.subflows.compute_TS' external_params=None options={'use_offline': True} prefect_config=None
[Errno 111] Connection refused
Cleaning :  /home/ubuntu/echoflow/echoflow/echoflow/tests/offli

/tmp/ipykernel_103521/3922359326.py:2: RuntimeWarning: coroutine 'Block.load' was never awaited
  data  = echoflow_start(dataset_config=dataset_config, pipeline_config=pipeline_config, options=options)


In [ ]:
data[0][0]